# SPE10 Demo - Single-core

In [1]:
#addprocs(4);
using DistributedArrays
using ForwardDiff
using StaticArrays
include("Stencil.jl")
include("stencilgmres.jl")
include("reservoirfunc.jl")
include("distributed.jl")
include("res3d.jl")
include("preconditioning.jl")
using BenchmarkTools

In [2]:
permdata = open("spe_perm.dat")
phidata  = open("spe_phi.dat")
raw1 = readdlm(phidata)
raw2 = readdlm(permdata)
close(permdata)
close(phidata)
porosity = reshape(raw1', 60, 220, 85)
lineperm = reshape(raw2', 3366000, 1)
kx = reshape(lineperm[1:1122000]', 60, 220, 85)
ky = reshape(lineperm[1122001:2244000]', 60, 220, 85)
kz = reshape(lineperm[2244001:end]', 60, 220, 85)
savepor = copy(porosity)
## Porosity pre-processing
for id in eachindex(porosity)
    if abs(porosity[id])<1e-3
        porosity[id] = 0.2
        kx[id], ky[id], kz[id] = 1e-10, 1e-10, 1e-10
    end
end
kraw = [SVector{3,Float64}([kx[i,j,k], ky[i,j,k], kz[i,j,k]]) for i in 1:60, j in 1:220, k in 1:85];

In [3]:
Nx = 30
Ny = 30
Nz = 30
offsetx = 1
offsety = 1
offsetz = 1;

In [4]:
p_ref = 14.7 
C_r   = 3e-6
ϕ_ref = 0.2
#=everywhere=# ϕ     = porosity[offsetx:(Nx+offsetx-1), offsety:(Ny+offsety-1), offsetz:(Nz+offsetz-1)]
S_wc, S_or = 0.2, 0.2
k_r_w(x)   = ((x-S_wc)/(1-S_wc-S_or))^2
k_r_o(x)   = (1 - (x-S_wc)/(1-S_wc-S_or))^2
p_cow(x)   = 0
C_water    = 3e-6
C_oil      = 3e-6
ρ_water(p) = 64.0*exp(C_water*(p-p_ref)) 
ρ_oil(p)   = 53.0*exp(C_oil*(p-p_ref))   
μ_water = 0.3 # cp
μ_oil   = 3.0; # cp
saveper = (kx.+ky.+kz)./3
saveper = saveper[offsetx:(Nx+offsetx-1), offsety:(Ny+offsety-1), offsetz:(Nz+offsetz-1)];

In [5]:
## 3d model
Lx = 1200
Ly = 2200
Lz = 170
Δt = 1.0
Tf = 2000.0
Δx = fill(Lx/Nx, Nx, Ny, Nz)
Δy = fill(Ly/Ny, Nx, Ny, Nz)
Δz = fill(Lz/Nz, Nx, Ny, Nz)
z  = fill(12000.0, Nx, Ny, Nz)
k = makegrid(kraw[offsetx:(Nx+offsetx-1), offsety:(Ny+offsety-1), offsetz:(Nz+offsetz-1)],7)
model = Reservoirmodel(Δt, Tf, (Δx, Δy, Δz), z, k, p_ref, C_r, ϕ_ref, ϕ, 
                k_r_w, k_r_o, p_cow, C_water, C_oil, ρ_water, ρ_oil, μ_water, μ_oil);

In [6]:
## Porosity proportional control
Total = 40.0*Nx
q = zeros(Nx, Ny, Nz, 2)
for i in (1,Nx), j in (1,Ny), k in 1:Nz
    q[i,j,k,2] = Total*(saveper[i,j,k]/sum(saveper[i,j,:]))/4
end
halfx, halfy = round(Int, Nx/2),round(Int, Ny/2)
for k in 1:Nz
    q[halfx,halfy,k,1] = -Total*(saveper[halfx,halfy,k]/sum(saveper[halfx,halfy,:]))
end 

In [7]:
init       = [SVector{2,Float64}([6000.0,0.2]) for i in 1:Nx, j in 1:Ny, k in 1:Nz]
testgrid   = makegrid(init, 7);
g_guess   = testgrid;

In [8]:
S   = getstencil(model, q, g_guess, testgrid)
res = getresidual(model, q, g_guess, testgrid);

# Single-core benchmark (residual, jacobian)

In [9]:
@btime getstencil(model, q, g_guess, testgrid);

  2.880 s (38097933 allocations: 2.06 GiB)


In [10]:
@btime getresidual(model, q, g_guess, testgrid);

  652.475 ms (18252022 allocations: 304.96 MiB)


In [11]:
function myfun(x)
    return x.^2
end

myfun (generic function with 1 method)

In [12]:
ForwardDiff.jacobian(myfun, rand(10))

10×10 Array{Float64,2}:
 1.64452  0.0       0.0       0.0      0.0      …  0.0      0.0      0.0    
 0.0      0.390997  0.0       0.0      0.0         0.0      0.0      0.0    
 0.0      0.0       0.850188  0.0      0.0         0.0      0.0      0.0    
 0.0      0.0       0.0       1.70125  0.0         0.0      0.0      0.0    
 0.0      0.0       0.0       0.0      1.49894     0.0      0.0      0.0    
 0.0      0.0       0.0       0.0      0.0      …  0.0      0.0      0.0    
 0.0      0.0       0.0       0.0      0.0         0.0      0.0      0.0    
 0.0      0.0       0.0       0.0      0.0         1.93977  0.0      0.0    
 0.0      0.0       0.0       0.0      0.0         0.0      1.95787  0.0    
 0.0      0.0       0.0       0.0      0.0         0.0      0.0      1.12034